# Task02
Needs Blackhole configured

In [8]:
# Import requirements
import spotipy
import pandas
import random
import numpy
import os

In [9]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 44100  # Sample rate
seconds = 24  # Duration of recording

In [10]:
# Import specific functions
from lib import *

In [11]:
# subject name/id
subject_id = 'sub-09'

In [12]:
# Setup Spotipy
scope = "user-library-read,user-read-playback-state,user-modify-playback-state"
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(scope=scope,client_id='253a1345850d4a5c94995c1159d286e4',
                                             client_secret='9a03e1bcb8124eed9b661991e84bf477',
                                             redirect_uri='http://127.0.0.1:9090'),
)

# choose deviceID (1st)
res = sp.devices()
devID = res['devices'][0]['id']

## Import subject-specific selection of songs

In [13]:
# read file
D = pandas.read_table(os.path.join('..','data','music_matrices',
                                   f"music_{subject_id}.tsv"),
                                index_col = "Emotion")

# extract list of emotions
emotionList = D.index.values

# Generate two new columns (start1MS and start2MS) in milliseconds for each song
for emotion in emotionList:
    D.loc[emotion,"startMS1"] = (int(D.start1[emotion][1:3])*60 + int(D.start1[emotion][4:6])) * 1000
    D.loc[emotion,"startMS2"] = (int(D.start2[emotion][1:3])*60 + int(D.start2[emotion][4:6])) * 1000

## Version 0.2a - 2 songs coupled by emotion

In [14]:
run_id = 'task-02a_run-1'

# create a new list with the same elements as emotionList, but in a random order
randomEmotionList = emotionList.copy()
random.shuffle(randomEmotionList)

# Initialize Dataframe to record timestamps
timeMatrix = pandas.DataFrame(columns=["emotion","trackID","timestamp0","before_music_start", "after_music_start", "before_noise_start", "after_noise_start"])

# set initial volume
sp.volume(60,devID)

# first baseline
timestamp0 = time.time()
sp.start_playback(uris=['spotify:track:7Ipe9FfRceDN51YMyd99pO'],position_ms=1000,device_id=devID)
wait_until(timestamp0 + 2)

# trial loop
for emotion in randomEmotionList:
    print(f"Now playing: {emotion}")

    for i in range(1,3):
        song = D[f"trackID{str(i)}"][emotion]
        start_time = D[f"startMS{str(i)}"][emotion].item()

        print(f"Now playing: {song} starting at {str(start_time)}")

        timestamp1 = time.time()

        sp.start_playback(
            uris=[f'spotify:track:{song}'],
            position_ms=start_time,
            device_id=devID,
        )
        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
        sd.wait()  # Wait until recording is finished
        write(os.path.join('..','data','wav_files',
                           f'{subject_id}_{emotion.replace(" ", "")}_{song}.wav'),
                         fs, myrecording)  # Save as WAV file

        timestamp2 = time.time()

        fade_in(sp,devID,duration=2,steps=5,t1=timestamp1)

        wait_until(timestamp1 + 22)

        fade_out(sp,devID,duration=2,steps=5,t1=timestamp1+22)

        timestamp3 = time.time()

        sp.start_playback(
            uris=['spotify:track:7Ipe9FfRceDN51YMyd99pO'],
            position_ms=1000,
            device_id=devID,
        )

        timestamp4 = time.time()

        wait_until(timestamp3 + 6)

    wait_until(timestamp3 + 6 + 12)

sp.pause_playback()

print('Done!')

Now playing: Peacefulness
Now playing: 6J2LdBN97cDWn0MLxYh9HB starting at 75000.0
Now playing: 1OU4E4HiVjdak0mL4blVWT starting at 62000.0
Now playing: Wonder
Now playing: 1tLp9KH0j1HoKdS1vnvkxO starting at 243000.0
Now playing: 0XXkQk83TjaGoPndxWLHKr starting at 48000.0
Now playing: Nostalgia
Now playing: 1TQ2UYCN7nhfj3cfcFC76V starting at 2000.0
Now playing: 4QTFU7BukoyDtyyemGPNCX starting at 9000.0
Now playing: Sadness
Now playing: 0pwnoUhy57HdGjj1zNzzNV starting at 81000.0
Now playing: 50Td3qilgs8BLtv8mHyT1t starting at 0.0
Now playing: Tension
Now playing: 70gOupwTsC5iq2x3i6Q8f3 starting at 0.0
Now playing: 1AUyzuh7CIr1yDYPTOgQQL starting at 194000.0
Now playing: Joyful activation
Now playing: 70t7Q6AYG6ZgTYmJWcnkUM starting at 50000.0
Now playing: 71ax09SvKzLb1mwlQeAYMU starting at 60000.0
Now playing: Transcendence
Now playing: 3nSglalUiJYQfUsnzvI5Px starting at 63000.0
Now playing: 5ky71GXSi2Rt2sgmbVGuqK starting at 90000.0
Now playing: Power
Now playing: 3kG6GPkDkV2RQTm9QdYN9Z 